# APPLIED DATA ANALYSIS PROJECT

In [1]:
import pandas as pd
import requests
import re

from bs4 import BeautifulSoup

In [2]:
URL_QS = 'https://en.wikipedia.org/wiki/List_of_former_European_colonies'
r = requests.get(URL_QS)
soup = BeautifulSoup(r.text, 'lxml')

In [3]:
colonizer_countries = []

# To find the name of all colonizer country of Europe, we take care about 2 things :
#  - If there is more than 2 spaces in the text -> isn't a country
#  - If there is less than 2 letters in the text -> isn't a country

for country in soup.findAll('p'):
    nb_space = sum(c.isspace() for c in country.text)
    
    if country.text not in colonizer_countries and nb_space < 3 and len(country.text) > 2:
        colonizer_countries.append(country.text)

# List Cleaning
colonizer_countries = [country.replace('\n', '') for country in colonizer_countries]

colonizer_countries

['Denmark',
 'France',
 'Netherlands',
 'Norway',
 'Portugal',
 'Russia',
 'Spain',
 'Sweden',
 'Brandenburg',
 'Britain',
 'Courland',
 'Denmark-Norway',
 'Knights of Malta',
 'Belgium',
 'Germany',
 'Italy',
 'Austria',
 'United Kingdom',
 'Germany',
 'Greece']

In [4]:
regions = []

for region in soup.find("ul").findAll('li'):
    regions.append(region.text[2:])

del regions[-1], regions[-1] 

regions

['North America',
 'West Indies and the Caribbean',
 'South America',
 'Africa',
 'Indian Ocean',
 'Middle East',
 'Indian Subcontinent',
 'Asia-Pacific',
 'Europe']

In [5]:
def check_country(df, colonized, colonizator, URL, first_check):
    
    URL_QS = 'https://en.wikipedia.org' + URL
    r = requests.get(URL_QS)
    soup = BeautifulSoup(r.text, 'lxml')

    if len(df[df["Colonized Country"] == colonized]) == 0:
    
        if not soup(text=re.compile('disestablished')) and not soup(text=re.compile('disestablishments')):  
            df.loc[len(df)+1] = [colonized, colonizator, URL]
        elif first_check:
            # Vérifier si ça juste pas un nom différent
            country = soup.find_all("b", text="Succeeded by")

            if country:
                a = country[0].parent.parent.next_sibling.next_sibling
                URL = a.find_all(style="border:0; padding:0; vertical-align:middle; text-align:right;")[0].a['href']
                country = a.find_all(style="border:0; padding:0; vertical-align:middle; text-align:right;")[0].text
                check_country(df, country, colonizator, URL, False)

In [6]:
# Only check for the XXX for now
colonizator = "France"

# Trouve tout les emplacements où on fait mension de "colonizator"
colonizator_index = soup.find_all("p", text=colonizator) 

colonization_df = pd.DataFrame(columns=['Colonized Country', 'Colonizator Country', "URL"])

for i in range (len(colonizator_index)):

    a = colonizator_index[i].next_sibling.next_sibling('a')

    for b in a:
        if b.text not in regions and len(b.attrs) < 3:
            check_country(colonization_df, b.text, colonizator, b['href'], True)
        
colonization_df


,Colonized Country,Colonizator Country,URL
1,Anguilla,France,/wiki/Anguilla
2,Dominica,France,/wiki/Dominica
3,Grenada,France,/wiki/Grenada
4,Haiti,France,/wiki/Haiti
5,Montserrat,France,/wiki/Montserrat
6,Nevis,France,/wiki/Nevis
7,Saint Christophe,France,/wiki/Saint_Kitts
8,Saint Vincent and the Grenadines,France,/wiki/Saint_Vincent_and_the_Grenadines
9,Sint Eustatius,France,/wiki/Sint_Eustatius
10,Tobago,France,/wiki/Tobago


In [7]:
# TO DO
# - Vérifier que les colonies n'appartiennent pas à un autre pays (genre)
# - Virer les territoires trop petit --> meilleur solution à priori